In [1]:
import torchvision.datasets as dataset
import torchvision.transforms as transform
import torchvision.transforms.functional as TF
from torch.utils.data import DataLoader
from model_search import Network_Multi_Path as Network
import matplotlib.pyplot as plt
class Trans(object):
    def __init__(self, transforms):
        self.scale = (0.08, 1.0)
        self.ratio = (3. / 4., 4. / 3.)
        self.size = (128,256)
        self.transforms = transforms
        self.target_resize = transform.Resize((16, 32))
        self.to_tensor = transform.ToTensor()

    def __call__(self, image, target):
        i, j, h, w = transform.RandomResizedCrop.get_params(image, self.scale, self.ratio)
        image = F.resized_crop(image, i, j, h, w, self.size)
        target = F.resized_crop(target, i, j, h, w, self.size)
        target = self.target_resize(target)
        image = self.to_tensor(image)
        target = self.to_tensor(target)
        target = target*255
        target = target.long()
        return image, target[0]

train_dataset = dataset.Cityscapes('/datasets/cityscapes/',split='train', mode='fine',
                     target_type='semantic',transforms=Compose([transform.RandomResizedCrop((128,256))]))

train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=4)

/home/jif055/model/utils/darts_utils.py:179: UserWarning: TensorRT (or pycuda) is not installed. compute_latency_ms_tensorrt() cannot be used.
  warnings.warn("TensorRT (or pycuda) is not installed. compute_latency_ms_tensorrt() cannot be used.")


In [2]:
imgs, labels = iter(train_dataloader).next()

In [3]:
model = Network(num_classes=35, layers=5)
model = model.cuda()

In [4]:
imgs = imgs.cuda()
labels = labels.cuda()


In [5]:
loss = model._loss(imgs, labels)

torch.Size([2, 96, 16, 32])
torch.Size([2, 96, 16, 32])
torch.Size([2, 96, 16, 32])


In [7]:
loss.backward()